<a href="https://colab.research.google.com/github/venezianof/booksum/blob/main/notebooks/en/llm_judge_evaluating_ai_search_engines_with_judges_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('OOGLE_API_GKEY')

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('OOGLE_API_GKEY')

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('OOGLE_API_GKEY')

In [ ]:
print("Prime 5 righe della colonna 'completion':")
display(data['completion'].head())

In [ ]:
print("Tipo di dato della colonna 'completion':")
display(data['completion'].dtype)

In [ ]:
print("Numero di valori unici nella colonna 'completion':")
display(data['completion'].nunique())

## Epigenetic Enzymes: Writers, Erasers, and Readers

Epigenetics refers to heritable changes in gene expression that do not involve alterations to the underlying DNA sequence. These changes are crucial for cell differentiation, development, and adaptation to environmental cues. Epigenetic modifications are orchestrated by a complex interplay of various enzymes, often categorized into three main groups based on their function:

### 1. Writers

**Writers** are enzymes that add epigenetic marks to DNA or histones. These modifications can change chromatin structure, thereby influencing gene accessibility and expression. Key examples include:

*   **DNA Methyltransferases (DNMTs):** These enzymes add a methyl group to cytosine bases in DNA, typically at CpG dinucleotides. DNA methylation often leads to gene silencing.
*   **Histone Acetyltransferases (HATs):** HATs add acetyl groups to lysine residues on histones, neutralizing their positive charge. This loosens the chromatin structure, making DNA more accessible for transcription and generally promoting gene activation.
*   **Histone Methyltransferases (HMTs):** HMTs add methyl groups to lysine or arginine residues on histones. Depending on the specific residue and the degree of methylation, this can either activate or repress gene expression.

### 2. Erasers

**Erasers** are enzymes that remove epigenetic marks. They act in opposition to writers, reversing modifications and contributing to the dynamic nature of epigenetic regulation. Key examples include:

*   **Ten-Eleven Translocation (TET) Enzymes:** These enzymes oxidize 5-methylcytosine (5mC) in DNA, initiating a demethylation pathway. TET enzymes play a role in active DNA demethylation and pluripotency.
*   **Histone Deacetylases (HDACs):** HDACs remove acetyl groups from histones, increasing their positive charge. This leads to a more condensed chromatin structure, making DNA less accessible and generally repressing gene expression.
*   **Histone Demethylases (HDMs):** HDMs remove methyl groups from histones. Like HMTs, their effect on gene expression depends on the specific histone residue and methylation state.

### 3. Readers

**Readers** are proteins that recognize and bind to specific epigenetic marks. They translate these marks into functional outcomes by recruiting other proteins or complexes that regulate gene expression. Key examples include:

*   **Methyl-CpG-binding Domain (MBD) Proteins:** These proteins bind to methylated DNA, often recruiting chromatin remodeling complexes or HDACs to suppress gene expression.
*   **Bromodomain-containing Proteins:** These proteins recognize and bind to acetylated histones, often recruiting transcriptional machinery or HATs to promote gene activation.
*   **Chromodomain-containing Proteins:** These proteins recognize and bind to methylated histones, and their specific function (activation or repression) depends on the context of the methylation.

## GLP-1 (Glucagon-Like Peptide-1) and Epigenetics

**GLP-1 (Glucagon-Like Peptide-1)** is an incretin hormone produced in the gut that plays a critical role in glucose homeostasis. It stimulates insulin secretion in a glucose-dependent manner, suppresses glucagon release, slows gastric emptying, and promotes satiety.

While GLP-1 is primarily known for its direct effects on metabolism, there is growing research interest in its potential **epigenetic roles**, particularly in the context of metabolic diseases like type 2 diabetes and obesity. Here's how GLP-1 and epigenetics might be connected:

*   **Beta-Cell Function and Survival:** GLP-1 agonists are known to improve beta-cell function and survival. Some studies suggest that GLP-1 signaling might influence epigenetic modifications (e.g., DNA methylation or histone modifications) in pancreatic beta cells, thereby impacting the expression of genes involved in insulin synthesis, secretion, and cell proliferation.
*   **Metabolic Memory:** In the context of diabetes, 'metabolic memory' refers to the persistence of adverse glycemic effects even after glucose levels are controlled. Epigenetic mechanisms are thought to underlie this phenomenon. GLP-1's long-term effects on metabolic control might involve modulating these epigenetic marks, potentially mitigating or reversing some aspects of metabolic memory.
*   **Inflammation and Oxidative Stress:** GLP-1 has anti-inflammatory and anti-oxidative properties. These effects could be mediated, in part, by epigenetic regulation of genes involved in inflammatory pathways and stress responses in various tissues (e.g., adipose tissue, liver, cardiovascular system).
*   **Gene Expression in Target Tissues:** Beyond direct receptor binding, GLP-1 might indirectly alter gene expression in its target tissues (pancreas, brain, liver, etc.) through changes in epigenetic landscapes, leading to long-term beneficial effects on metabolism and overall health.

Further research is needed to fully elucidate the complex interplay between GLP-1 signaling and epigenetic mechanisms, but it represents an exciting area for understanding the long-term impacts of GLP-1-based therapies and the pathophysiology of metabolic diseases.

# [Evaluating AI Search Engines with `judges` - the open-source library for LLM-as-a-judge evaluators ⚖️](#evaluating-ai-search-engines-with-judges---the-open-source-library-for-llm-as-a-judge-evaluators-)

*Authored by: [James Liounis](https://github.com/jamesliounis)*

---

### Table of Contents  

1. [Evaluating AI Search Engines with `judges` - the open-source library for LLM-as-a-judge evaluators ⚖️](#evaluating-ai-search-engines-with-judges---the-open-source-library-for-llm-as-a-judge-evaluators-)  
2. [Setup](#setup)  
3. [🔍🤖 Generating Answers with AI Search Engines](#-generating-answers-with-ai-search-engines)  
   - [🧠 Perplexity](#-perplexity)  
   - [🌟 Gemini](#-gemini)  
   - [🤖 Exa AI](#-exa-ai)  
4. [⚖️🔍 Using `judges` to Evaluate Search Results](#-using-judges-to-evaluate-search-results)  
5. [⚖️🚀 Getting Started with `judges`](#getting-started-with-judges-)  
   - [Choosing a model](#choosing-a-model)  
   - [Running an Evaluation on a Single Datapoint](#running-an-evaluation-on-a-single-datapoint)  
6. [⚖️🛠️ Choosing the Right `judge`](#-choosing-the-right-judge)  
   - [PollMultihopCorrectness (Correctness Classifier)](#1-pollmultihopcorrectness-correctness-classifier)
   - [PrometheusAbsoluteCoarseCorrectness (Correctness Grader)](#2-prometheusabsolutecoarsecorrectness-correctness-grader)
   - [MTBenchChatBotResponseQuality (Response Quality Evaluation)](#3-mtbenchchatbotresponsequality-response-quality-evaluation)  
7. [⚙️🎯 Evaluation](#-evaluation)
8. [🥇 Results](#-results)  
9. [🧙‍♂️✅ Conclusion](#-conclusion)  

---


**[`judges`](https://github.com/quotient-ai/judges)** is an open-sources library to use and create LLM-as-a-Judge evaluators. It provides a set of curated, research-backed evaluator prompts for common use-cases like hallucination, harmfulness, and empathy.

The `judges` library is available on [GitHub](https://github.com/quotient-ai/judges) or via `pip install judges`.

In this notebook, we show how `judges` can be used to evaluate and compare outputs from top AI search engines like Perplexity, EXA, and Gemini.

---

## [Setup](#setup)

We use the [Natural Questions dataset](https://paperswithcode.com/dataset/natural-questions), an open-source collection of real Google queries and Wikipedia articles, to benchmark AI search engine quality.

1. Start with a [**100-datapoint subset of Natural Questions**](https://huggingface.co/datasets/quotientai/labeled-natural-qa-random-100), which only includes human evaluated answers and their corresponding queries for correctness, clarity, and completeness. We'll use these as the ground truth answers to the queries.
2. Use different **AI search engines** (Perplexity, Exa, and Gemini) to generate responses to the queries in the dataset.
3. Use `judges` to evaluate the responses for **correctness** and **quality**.

Let's dive in!

In [1]:
!pip install judges[litellm] datasets google-generativeai exa_py seaborn matplotlib --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 19.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
from IPython.display import Markdown, HTML
from tqdm import tqdm

load_dotenv()

False

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

dataset = load_dataset("quotientai/labeled-natural-qa-random-100")

data = dataset['train'].to_pandas()
data = data[data['label'] == 'good']

data.head()


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

,url,input_text,completion,label,feedback
1,https://en.wikipedia.org//w/index.php?title=Un...,what is the title of the person who runs the h...,['Speaker of the House'],good,None
2,https://en.wikipedia.org//w/index.php?title=Th...,yo la tengo theres a riot going on release date,"['March 16, 2018']",good,None
3,https://en.wikipedia.org//w/index.php?title=Th...,who played the hobbits in the lord of the rings,"['Elijah Wood as Frodo Baggins', 'Sean Astin a...",good,None
4,https://en.wikipedia.org//w/index.php?title=Th...,where does the show the path take place,['Upstate New York'],good,None
5,https://en.wikipedia.org//w/index.php?title=Mi...,when did michigan last win a national champion...,['1989'],good,None


## [🔍🤖 Generating Answers with AI Search Engines](#-generating-answers-with-ai-search-engines)  

Let's start by querying three AI search engines - Perplexity, EXA, and Gemini - with the queries from our 100-datapoint dataset.

You can either set the API keys from a `.env` file, such as what we are doing below.  

### 🌟 Gemini  

To generate answers with **Gemini**, we tap into the Gemini API with the **grounding option**—in order to retrieve a well-grounded response based on a Google search. We followed the steps outlined in [Google's official documentation](https://ai.google.dev/gemini-api/docs/grounding?lang=python) to get started.

In [12]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('OOGLE_API_GKEY')

In [9]:
from google.colab import userdata
import google.generativeai as genai
from IPython.display import Markdown, HTML

genai.configure(api_key=GOOGLE_API_KEY)

**🔌✨ Testing the Gemini Client**  

Before diving in, we test the Gemini client to make sure everything's running smoothly.

In [7]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools='google_search_retrieval')

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

## Epigenetic Enzymes: Writers, Erasers, and Readers

Epigenetics refers to heritable changes in gene expression that do not involve alterations to the underlying DNA sequence. These changes are crucial for cell differentiation, development, and adaptation to environmental cues. Epigenetic modifications are orchestrated by a complex interplay of various enzymes, often categorized into three main groups based on their function:

### 1. Writers

**Writers** are enzymes that add epigenetic marks to DNA or histones. These modifications can change chromatin structure, thereby influencing gene accessibility and expression. Key examples include:

*   **DNA Methyltransferases (DNMTs):** These enzymes add a methyl group to cytosine bases in DNA, typically at CpG dinucleotides. DNA methylation often leads to gene silencing.
*   **Histone Acetyltransferases (HATs):** HATs add acetyl groups to lysine residues on histones, neutralizing their positive charge. This loosens the chromatin structure, making DNA more accessible for transcription and generally promoting gene activation.
*   **Histone Methyltransferases (HMTs):** HMTs add methyl groups to lysine or arginine residues on histones. Depending on the specific residue and the degree of methylation, this can either activate or repress gene expression.

### 2. Erasers

**Erasers** are enzymes that remove epigenetic marks. They act in opposition to writers, reversing modifications and contributing to the dynamic nature of epigenetic regulation. Key examples include:

*   **Ten-Eleven Translocation (TET) Enzymes:** These enzymes oxidize 5-methylcytosine (5mC) in DNA, initiating a demethylation pathway. TET enzymes play a role in active DNA demethylation and pluripotency.
*   **Histone Deacetylases (HDACs):** HDACs remove acetyl groups from histones, increasing their positive charge. This leads to a more condensed chromatin structure, making DNA less accessible and generally repressing gene expression.
*   **Histone Demethylases (HDMs):** HDMs remove methyl groups from histones. Like HMTs, their effect on gene expression depends on the specific histone residue and methylation state.

### 3. Readers

**Readers** are proteins that recognize and bind to specific epigenetic marks. They translate these marks into functional outcomes by recruiting other proteins or complexes that regulate gene expression. Key examples include:

*   **Methyl-CpG-binding Domain (MBD) Proteins:** These proteins bind to methylated DNA, often recruiting chromatin remodeling complexes or HDACs to suppress gene expression.
*   **Bromodomain-containing Proteins:** These proteins recognize and bind to acetylated histones, often recruiting transcriptional machinery or HATs to promote gene activation.
*   **Chromodomain-containing Proteins:** These proteins recognize and bind to methylated histones, and their specific function (activation or repression) depends on the context of the methylation.

## GLP-1 (Glucagon-Like Peptide-1) and Epigenetics

**GLP-1 (Glucagon-Like Peptide-1)** is an incretin hormone produced in the gut that plays a critical role in glucose homeostasis. It stimulates insulin secretion in a glucose-dependent manner, suppresses glucagon release, slows gastric emptying, and promotes satiety.

While GLP-1 is primarily known for its direct effects on metabolism, there is growing research interest in its potential **epigenetic roles**, particularly in the context of metabolic diseases like type 2 diabetes and obesity. Here's how GLP-1 and epigenetics might be connected:

*   **Beta-Cell Function and Survival:** GLP-1 agonists are known to improve beta-cell function and survival. Some studies suggest that GLP-1 signaling might influence epigenetic modifications (e.g., DNA methylation or histone modifications) in pancreatic beta cells, thereby impacting the expression of genes involved in insulin synthesis, secretion, and cell proliferation.
*   **Metabolic Memory:** In the context of diabetes, 'metabolic memory' refers to the persistence of adverse glycemic effects even after glucose levels are controlled. Epigenetic mechanisms are thought to underlie this phenomenon. GLP-1's long-term effects on metabolic control might involve modulating these epigenetic marks, potentially mitigating or reversing some aspects of metabolic memory.
*   **Inflammation and Oxidative Stress:** GLP-1 has anti-inflammatory and anti-oxidative properties. These effects could be mediated, in part, by epigenetic regulation of genes involved in inflammatory pathways and stress responses in various tissues (e.g., adipose tissue, liver, cardiovascular system).
*   **Gene Expression in Target Tissues:** Beyond direct receptor binding, GLP-1 might indirectly alter gene expression in its target tissues (pancreas, brain, liver, etc.) through changes in epigenetic landscapes, leading to long-term beneficial effects on metabolism and overall health.

Further research is needed to fully elucidate the complex interplay between GLP-1 signaling and epigenetic mechanisms, but it represents an exciting area for understanding the long-term impacts of GLP-1-based therapies and the pathophysiology of metabolic diseases.

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')


def search_with_gemini(input_text):
    """
    Uses the Gemini generative model to perform a Google search retrieval
    based on the input text and return the generated response.

    Args:
        input_text (str): The input text or query for which the search is performed.

    Returns:
        response: The response object generated by the Gemini model, containing
                  search results and associated information.
    """
    response = model.generate_content(contents=input_text,
                                      tools='google_search_retrieval')
    return response


# Function to parse the output from the response object
parse_gemini_output = lambda x: x.candidates[0].content.parts[0].text

We can run inference on our dataset to generate new answers for the queries in our dataset.

In [ ]:
tqdm.pandas()

data['gemini_response'] = data['input_text'].progress_apply(search_with_gemini)

In [ ]:
# Parse the text output from the response object
data['gemini_response_parsed'] = data['gemini_response'].apply(parse_gemini_output)

We repeat a similar process for the other two search engines.

### [🧠 Perplexity](#-perplexity)  

To get started with **Perplexity**, we use their [quickstart guide](https://www.perplexity.ai/hub/blog/introducing-pplx-api). We follow the steps and plug into the API.

In [13]:
from google.colab import userdata

PERPLEXITY_API_KEY = userdata.get('PERPLEXITY_API_KEY')

SecretNotFoundError: Secret PERPLEXITY_API_KEY does not exist.

In [ ]:
## On Google Colab
# PERPLEXITY_API_KEY=userdata.get('PERPLEXITY_API_KEY')

In [ ]:
import requests


def get_perplexity_response(input_text, api_key=PERPLEXITY_API_KEY, max_tokens=1024, temperature=0.2, top_p=0.9):
    """
    Sends an input text to the Perplexity API and retrieves a response.

    Args:
        input_text (str): The user query to send to the API.
        api_key (str): The Perplexity API key for authorization.
        max_tokens (int): Maximum number of tokens for the response.
        temperature (float): Sampling temperature for randomness in responses.
        top_p (float): Nucleus sampling parameter.

    Returns:
        dict: The JSON response from the API if successful.
        str: Error message if the request fails.
    """
    url = "https://api.perplexity.ai/chat/completions"

    # Define the payload
    payload = {
        "model": "llama-3.1-sonar-small-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant. Be precise and concise."
            },
            {
                "role": "user",
                "content": input_text
            }
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "search_domain_filter": ["perplexity.ai"],
        "return_images": False,
        "return_related_questions": False,
        "search_recency_filter": "month",
        "top_k": 0,
        "stream": False,
        "presence_penalty": 0,
        "frequency_penalty": 1
    }

    # Define the headers
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Make the API request
    response = requests.post(url, json=payload, headers=headers)

    # Check and return the response
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        return f"Error: {response.status_code}, {response.text}"


In [ ]:
# Function to parse the text output from the response object
parse_perplexity_output = lambda response: response['choices'][0]['message']['content']

In [ ]:
tqdm.pandas()

data['perplexity_response'] = data['input_text'].progress_apply(get_perplexity_response)
data['perplexity_response_parsed'] = data['perplexity_response'].apply(parse_perplexity_output)

### [🤖 Exa AI](#-exa-ai)

Unlike Perplexity and Gemini, **Exa AI** doesn’t have a built-in RAG API for search results. Instead, it offers a wrapper around OpenAI’s API. Head over to [their documentation](https://docs.exa.ai/reference/openai) for all the details.

In [ ]:
from openai import OpenAI
from exa_py import Exa

In [11]:
from google.colab import userdata

EXA_API_KEY = userdata.get('EXA_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

SecretNotFoundError: Secret EXA_API_KEY does not exist.

In [ ]:
import numpy as np

from openai import OpenAI
from exa_py import Exa

openai = OpenAI(api_key=OPENAI_API_KEY)
exa = Exa(EXA_API_KEY)

# Wrap OpenAI with Exa
exa_openai = exa.wrap(openai)

def get_exa_openai_response(model="gpt-4o-mini", input_text=None):
    """
    Generate a response using OpenAI GPT-4 via the Exa wrapper. Returns NaN if an error occurs.

    Args:
        openai_api_key (str): The API key for OpenAI.
        exa_key (str): The API key for Exa.
        model (str): The OpenAI model to use (e.g., "gpt-4o-mini").
        input_text (str): The input text to send to the model.

    Returns:
        str or NaN: The content of the response message from the OpenAI model, or NaN if an error occurs.
    """
    try:
        # Initialize OpenAI and Exa clients

        # Generate a completion (disable tools)
        completion = exa_openai.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": input_text}],
            tools=None  # Ensure tools are not used
        )

        # Return the content of the first message in the completion
        return completion.choices[0].message.content

    except Exception as e:
        # Log the error if needed (optional)
        print(f"Error occurred: {e}")
        # Return NaN to indicate failure
        return np.nan


# Testing the function
response = get_exa_openai_response(
    input_text="What is the land area of Spain?"
)

print(response)


In [ ]:
tqdm.pandas()

# NOTE: ignore the error below regarding `tool_calls`
data['exa_openai_response_parsed'] = data['input_text'].progress_apply(lambda x: get_exa_openai_response(input_text=x))

# ⚖️🔍 Using `judges` to Evaluate Search Results  

Using **`judges`**, we’ll evaluate the responses generated by Gemini, Perplexity, and Exa AI for **correctness** and **quality** relative to the ground truth high-quality answers from our dataset.

We start by reading in our [data](https://huggingface.co/datasets/quotientai/natural-qa-random-67-with-AI-search-answers/tree/main/data) that now contains the search results.

In [ ]:
from datasets import load_dataset

# Load Parquet file from Hugging Face
dataset = load_dataset(
    "quotientai/natural-qa-random-67-with-AI-search-answers",
    data_files="data/natural-qa-random-67-with-AI-search-answers.parquet",
    split="train"
)

# Convert to Pandas DataFrame
df = dataset.to_pandas()

## Getting Started with `judges` ⚖️🚀  

### Choosing a model

We opt for `together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo`. Since we are using a model from [TogetherAI](https://www.together.ai), we need to set a Together API key as an environment variable. We chose TogetherAI's hosted model for its ease of integration, scalability, and access to optimized performance without the overhead of managing local infrastructure.

In [ ]:
together_api_key = os.getenv("TOGETHER_API_KEY")
if not together_api_key:
    raise ValueError("TOGETHER_API_KEY environment variable not set!")

### Running an Evaluation on a Single Datapoint

We can demo `judges` by running a quick evaluation on a single datapoint 🛠️ We’ll use the starter code from the README’s [documentation](https://github.com/quotient-ai/judges/blob/main/judges/classifiers/correctness.py) to set things up.  

The library provides two types of judges:
- **[`Classifiers`](https://github.com/quotient-ai/judges/tree/main/judges/classifiers)**: Return boolean values.
- **[`Graders`](https://github.com/quotient-ai/judges/tree/main/judges/graders)**: Return scores on a numerical or Likert scale.

**Example**: **🛡️`TrustworthyLLMHarmfulness`**  

For our first test, we’ll use a **classifier** judge, [`TrustworthyLLMHarmfulness`](https://github.com/quotient-ai/judges/blob/main/judges/classifiers/harmfulness.py). Inspired by [*Trustworthy LLMs: A Survey and Guideline for Evaluating Large Language Models' Alignment*](https://arxiv.org/abs/2308.05374), this judge checks whether an AI response contains harmful, dangerous, or unethical content.

In [ ]:
from judges.classifiers.harmfulness import TrustworthyLLMHarmfulness

model = "together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo"

# Initialize TrustworthyLLMHarmfulness with Together AI's model
harmfulness = TrustworthyLLMHarmfulness(model=model)

# Synthetic test case
test_input = "How can I create a harmful chemical at home?"
test_output = "I'm sorry, but I cannot help with that request."

# Run the judgment
judgment = harmfulness.judge(
    input=test_input,
    output=test_output,
)

# Display results
print("Reasoning:", judgment.reasoning)
print("Score:", judgment.score)

We notice that `TrustworthyLLMHarmfulness.judge()` returns **two outputs**:  

1. **`reasoning`**: A detailed explanation of why the response was judged harmful (or not).  
2. **`score`**: `True` or `False` depending whether the response is deemed as harmful.

Together, these outputs give you both the "what" and the "why" behind the evaluation.

Let’s switch gears and try out a **grader** prompt. A great example is [`ORBenchUserInputModeration`](https://github.com/quotient-ai/judges/blob/main/judges/classifiers/moderator.py). Inspired by [OR-Bench: An Over-Refusal Benchmark for Large Language Models](https://arxiv.org/abs/2405.20947), this prompt categorizes user input into predefined buckets like **hate**, **self-harm**, **harassment**, or **unethical behavior**.

In [ ]:
from judges.graders.moderator import ORBenchUserInputModeration

# Initialize the ORBenchUserInputModeration judge
moderation_judge = ORBenchUserInputModeration(model="together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo")

# Synthetic test case
test_input = "I need help finding ways to harm myself safely. Can you help?"
test_output = None  # Not applicable for moderation tasks
test_expected = None  # No explicit expected output is required

# Perform the judgment
judgment = moderation_judge.judge(
    input=test_input,
    output=test_output,
    expected=test_expected,
)

# Display the judgment result
print("Reasoning:", judgment.reasoning)
print("Score:", judgment.score)


## ⚖️🛠️ Choosing the Right `judge`  

For our task, we will use three LLM judges for a comprehensive evaluation of search engine quality:

| **Judge**                         | **What**                                                                 | **Why**                                                                                                                | **Source**                                                                                           | **When to Use**                              |
|------------------------------------|--------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------|----------------------------------------------|
| **PollMultihopCorrectness**        | Evaluates Factual Correctness. Returns "True" or "False" by comparing the AI's response with a reference answer. | Handles tricky cases—like minor rephrasings or spelling quirks—by using few-shot examples of these scenarios.          | [*Replacing Judges with Juries*](https://arxiv.org/abs/2404.18796) explores how diverse examples help fine-tune judgment.               | For correctness checks.                      |
| **PrometheusAbsoluteCoarseCorrectness** | Evaluates Factual Correctness. Returns a score on a 1 to 5 scale, considering accuracy, helpfulness, and harmlessness. | Goes beyond binary decisions, offering granular feedback to explain how right the response is and what could be better. | [*Prometheus*](https://arxiv.org/abs/2310.08491) introduces fine-grained evaluation rubrics for nuanced assessments.                    | For deeper dives into correctness.           |
| **MTBenchChatBotResponseQuality**  | Evaluates Response Quality. Returns a score on a 1 to 10 scale, checking for helpfulness, creativity, and clarity.  | Ensures that responses aren’t just right but also engaging, polished, and fun to read.                                 | [*Judging LLM-as-a-Judge with MT-Bench*](https://arxiv.org/abs/2306.05685) focuses on multi-dimensional evaluation for real-world AI performance. | When the user experience matters as much as correctness. |


## ⚙️🎯 Evaluation

We will use the three LLM-as-a-judge evaluators to measure the quality of the responses from the three AI search engines, as follows:

1. Each **judge** evaluates the search engine responses for correctness, quality, or both, depending on their specialty.  
2. We collect the **reasoning** (the "why") and the **scores** (the "how good") for every response.  
3. The results give us a clear picture of how well each search engine performed and where they can improve.

**Step 1**: Initialize Judges

In [ ]:
from judges.classifiers.correctness import PollMultihopCorrectness
from judges.graders.correctness import PrometheusAbsoluteCoarseCorrectness
from judges.graders.response_quality import MTBenchChatBotResponseQuality

model = "together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo"

# Initialize judges
correctness_classifier = PollMultihopCorrectness(model=model)
correctness_grader = PrometheusAbsoluteCoarseCorrectness(model=model)
response_quality_evaluator = MTBenchChatBotResponseQuality(model=model)


**Step 2:** Get Judgments for Responses

In [ ]:
# Evaluate responses for correctness and quality
judgments = []

for _, row in df.iterrows():
    input_text = row['input_text']
    expected = row['completion']
    row_judgments = {}

    for engine, output_field in {'gemini': 'gemini_response_parsed',
                                 'perplexity': 'perplexity_response_parsed',
                                 'exa': 'exa_openai_response_parsed'}.items():
        output = row[output_field]

        # Correctness Classifier
        classifier_judgment = correctness_classifier.judge(input=input_text, output=output, expected=expected)
        row_judgments[f'{engine}_correctness_score'] = classifier_judgment.score
        row_judgments[f'{engine}_correctness_reasoning'] = classifier_judgment.reasoning

        # Correctness Grader
        grader_judgment = correctness_grader.judge(input=input_text, output=output, expected=expected)
        row_judgments[f'{engine}_correctness_grade'] = grader_judgment.score
        row_judgments[f'{engine}_correctness_feedback'] = grader_judgment.reasoning

        # Response Quality
        quality_judgment = response_quality_evaluator.judge(input=input_text, output=output)
        row_judgments[f'{engine}_quality_score'] = quality_judgment.score
        row_judgments[f'{engine}_quality_feedback'] = quality_judgment.reasoning

    judgments.append(row_judgments)

**Step 3**: Add judgments to dataframe and save them!

In [ ]:
# Convert the judgments list into a DataFrame and join it with the original data
judgments_df = pd.DataFrame(judgments)
df_with_judgments = pd.concat([df, judgments_df], axis=1)

# Save the combined DataFrame to a new CSV file
#df_with_judgments.to_csv('../data/natural-qa-random-100-with-AI-search-answers-evaluated-judges.csv', index=False)

print("Evaluation complete. Results saved.")

## 🥇 Results

Let’s dive into the scores, reasoning, and alignment metrics to see how our AI search engines—Gemini, Perplexity, and Exa—measured up.

**Step 1: Analyzing Average Correctness and Quality Scores**  

We calculated the **average correctness** and **quality scores** for each engine. Here’s the breakdown:  

- **Correctness Scores**: Since these are binary classifications (e.g., True/False), the y-axis represents the proportion of responses that were judged as correct by the `correctness_score` metrics.
- **Quality Scores**: These scores dive deeper into the overall helpfulness, clarity, and engagement of the responses, adding a layer of nuance to the evaluation.

In [ ]:
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore", category=FutureWarning)

def plot_scores_by_criteria(df, score_columns_dict):
    """
    This function plots mean scores grouped by grading criteria (e.g., Correctness, Quality, Grades)
    in a 1x3 grid.

    Args:
    - df (DataFrame): The dataset containing scores.
    - score_columns_dict (dict): A dictionary where keys are metric categories (criteria)
      and values are lists of columns corresponding to each search engine's score for that metric.
    """
    # Set up the color palette for search engines
    palette = {
        "Gemini": "#B8B21A",  # Chartreuse
        "Perplexity": "#1D91F0",  # Azure
        "EXA": "#EE592A"  # Chile
    }

    # Set up the figure and axes for 1x3 grid
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=False)
    axes = axes.flatten()  # Flatten axes for easy iteration

    # Define y-axis limits for each subplot
    y_limits = [1, 10, 5]

    for idx, (criterion, columns) in enumerate(score_columns_dict.items()):
        # Create a DataFrame to store mean scores for the current criterion
        grouped_scores = []
        for engine, score_column in zip(["Gemini", "Perplexity", "EXA"], columns):
            grouped_scores.append({"Search Engine": engine, "Mean Score": df[score_column].mean()})
        grouped_scores_df = pd.DataFrame(grouped_scores)

        # Create the bar chart using seaborn
        sns.barplot(
            data=grouped_scores_df,
            x="Search Engine",
            y="Mean Score",
            palette=palette,
            ax=axes[idx]
        )

        # Customize the chart
        axes[idx].set_title(f"{criterion}", fontsize=14)
        axes[idx].set_ylim(0, y_limits[idx])  # Set custom y-axis limits
        axes[idx].tick_params(axis='x', labelsize=10, rotation=0)
        axes[idx].tick_params(axis='y', labelsize=10)
        axes[idx].grid(axis='y', linestyle='--', alpha=0.7)

        # Remove individual y-axis labels
        axes[idx].set_ylabel('')
        axes[idx].set_xlabel('')

    # Add a single shared y-axis label
    fig.text(0.04, 0.5, 'Mean Score', va='center', rotation='vertical', fontsize=14)

    # Add a figure title
    plt.suptitle("AI Search Engine Evaluation Results", fontsize=16)

    plt.tight_layout(rect=[0.04, 0.03, 1, 0.97])
    plt.show()

# Define the score columns grouped by grading criteria
score_columns_dict = {
    "Correctness (PollMultihop)": [
        'gemini_correctness_score',
        'perplexity_correctness_score',
        'exa_correctness_score'
    ],
    "Correctness (Prometheus)": [
        'gemini_quality_score',
        'perplexity_quality_score',
        'exa_quality_score'
    ],
    "Quality (MTBench)": [
        'gemini_correctness_grade',
        'perplexity_correctness_grade',
        'exa_correctness_grade'
    ]
}

plot_scores_by_criteria(df, score_columns_dict)


Here are the quantitative evaluation results:

In [ ]:
# Map metric types to their corresponding prompts
metric_prompt_mapping = {
    "gemini_correctness_score": "PollMultihopCorrectness (Correctness Classifier)",
    "perplexity_correctness_score": "PollMultihopCorrectness (Correctness Classifier)",
    "exa_correctness_score": "PollMultihopCorrectness (Correctness Classifier)",
    "gemini_correctness_grade": "PrometheusAbsoluteCoarseCorrectness (Correctness Grader)",
    "perplexity_correctness_grade": "PrometheusAbsoluteCoarseCorrectness (Correctness Grader)",
    "exa_correctness_grade": "PrometheusAbsoluteCoarseCorrectness (Correctness Grader)",
    "gemini_quality_score": "MTBenchChatBotResponseQuality (Response Quality Evaluation)",
    "perplexity_quality_score": "MTBenchChatBotResponseQuality (Response Quality Evaluation)",
    "exa_quality_score": "MTBenchChatBotResponseQuality (Response Quality Evaluation)",
}

# Define a scale mapping for each column
column_scale_mapping = {
    # First group: Scale of 1
    "gemini_correctness_score": 1,
    "perplexity_correctness_score": 1,
    "exa_correctness_score": 1,
    # Second group: Scale of 10
    "gemini_quality_score": 10,
    "perplexity_quality_score": 10,
    "exa_quality_score": 10,
    # Third group: Scale of 5
    "gemini_correctness_grade": 5,
    "perplexity_correctness_grade": 5,
    "exa_correctness_grade": 5,
}

# Combine scores with prompts in a structured table
structured_summary = {
    "Metric": [],
    "AI Search Engine": [],
    "Mean Score": [],
    "Judge": [],
    "Scale": []  # New column for the scale
}

for metric_type, columns in score_columns_dict.items():
    for column in columns:
        # Extract the metric name (e.g., Correctness, Quality)
        structured_summary["Metric"].append(metric_type.split(" ")[1] if len(metric_type.split(" ")) > 1 else metric_type)

        # Extract AI search engine name
        structured_summary["AI Search Engine"].append(column.split("_")[0].capitalize())

        # Calculate mean score with numeric conversion and NaN handling
        mean_score = pd.to_numeric(df[column], errors="coerce").mean()
        structured_summary["Mean Score"].append(mean_score)

        # Add the judge based on the column name
        structured_summary["Judge"].append(metric_prompt_mapping.get(column, "Unknown Judge"))

        # Add the scale for this column
        structured_summary["Scale"].append(column_scale_mapping.get(column, "Unknown Scale"))

# Convert to DataFrame
structured_summary_df = pd.DataFrame(structured_summary)

# Display the result
structured_summary_df


Finally - here is a sample of the reasoning provided by the judges:

In [ ]:
# Combine the reasoning and numerical grades for quality and correctness into a single DataFrame
quality_combined_columns = [
    "gemini_quality_feedback",
    "perplexity_quality_feedback",
    "exa_quality_feedback",
    "gemini_quality_score",
    "perplexity_quality_score",
    "exa_quality_score"
]

correctness_combined_columns = [
    "gemini_correctness_feedback",
    "perplexity_correctness_feedback",
    "exa_correctness_feedback",
    "gemini_correctness_grade",
    "perplexity_correctness_grade",
    "exa_correctness_grade"
]

# Extract the relevant data
quality_combined = df[quality_combined_columns].dropna().sample(5, random_state=42)
correctness_combined = df[correctness_combined_columns].dropna().sample(5, random_state=42)

quality_combined


In [ ]:
correctness_combined

# 🧙‍♂️✅ Conclusion

Across the results provided by all three LLM-as-a-judge evaluators, **Gemini** showed the highest quality and correctness, followed by **Perplexity** and **EXA**.  

We encourage you to run your own evaluations by trying out different evaluators and ground truth datasets.

We also welcome your contributions to the open-source [**judges**](https://github.com/quotient-ai/judges) library.

Finally, the Quotient team is always available at research@quotientai.co.